In [147]:
ua = {"User-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36 Edg/96.0.1054.57"}
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
import pickle
import re
import requests
import time

In [105]:
url =  "https://www.nankankeiba.com"
course_d = { "浦和": "18", "船橋": "19", "大井": "20", "川崎": "21" }

def get_soup(url):
    res = requests.get(url, headers=ua)
    
    return BeautifulSoup(res.content, "html.parser")

def get_dfs(url):
    soup = get_soup(url)
    dfs = []
    if soup.find("table"):
        dfs = pd.io.html.read_html(soup.prettify())
    else:
        print(f"it's no table! {url}")
    return dfs

In [106]:
date = "0317"
course = "浦和"
hold_code = "1304"
race = 3

hold = date + course_d[course] + hold_code
race_url = url + "/race_info/2022" + hold + str(race).rjust(2, "0") + ".do"
dfs = get_dfs(race_url)
df = dfs[-1]
df

,枠 番,馬 番,馬名 生年月日,性齢 毛色,馬体重 増減,負担 重量,騎手名 (所属),父 母,調教師 （所属）,馬主 生産牧場,単勝 オッズ,お気に入り馬
0,1,1,バローロフレイバー 16.3.16,牡6 芦毛,509 ＋5,56.0,今野忠成 (川崎),クロフネ シャインプレジャー,繁田健一 (浦和),尾田信夫 船越伸也,writeOdds(1);,NaN
1,2,2,セコイア 17.4.7(中同名),牝5 栗毛,465 -11,54.0,半澤慶実 (浦和),カレンブラックヒル ミスベルツリー,宇野木博徳 (浦和),福田達也 見上牧場,writeOdds(2);,NaN
2,3,3,ドンキースタイル 16.5.5(中同名),牡6 黒鹿,475 -4,56.0,山崎誠士 (川崎),ルーラーシップ ケイティーズミスト,山崎尋美 (川崎),二木英実 富田牧場,writeOdds(3);,NaN
3,4,4,トキノメガミ 15.3.17(中同名),牝7 栗毛,470 ＋4,53.0,篠谷葵 ☆ (船橋),トビーズコーナー マーベラスローズ,山越光 (浦和),松谷翔太 中山牧場,writeOdds(4);,NaN
4,5,5,サマーボーイ 15.4.7(中同名),牡7 鹿毛,504 ＋10,56.0,和田譲治 (大井),キンシャサノキセキ ノブクィーン,小久保智 (浦和),前田亘輝 千代田牧場,writeOdds(5);,NaN
5,5,6,キョウエイミーレス 16.3.7(中同名),牡6 鹿毛,506 -10,56.0,笹川翼 (大井),ジョーカプチーノ キョウエイエルメス,内野健二 (浦和),堀内一夫 藤沢牧場,writeOdds(6);,NaN
6,6,7,アスラーダ 17.3.7,牡5 鹿毛,474 ＋2,56.0,張田昂 (船橋),サムライハート ブラックスワン,横山保 (浦和),グリーンウッド（株） 織田正敏,writeOdds(7);,NaN
7,6,8,サワセンミラクル 17.4.8,牡5 青鹿,453 -4,56.0,橋本直哉 (浦和),パドトロワ レイナレイナ,酒井一則 (浦和),杉浦和也 グラストレーニングセンター,writeOdds(8);,NaN
8,7,9,ウィローグラウス 16.4.15(中同名),牝6 鹿毛,475 -3,54.0,臼井健太郎 (船橋),スマートファルコン コスモランゲル,矢内博 (船橋),田村義徳 ビッグレッドファーム,writeOdds(9);,NaN
9,7,10,グランピー 16.3.18,牝6 黒鹿,416 -3,54.0,伊藤裕人 (川崎),ロージズインメイ マジックミラージュ,久保勇 (川崎),金野博幸 下河辺牧場,writeOdds(10);,NaN


In [107]:
soup = get_soup(race_url)
soup


<?xml version="1.0" encoding="Shift_JIS"?>
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html lang="ja" xml:lang="ja" xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="ja" http-equiv="content-language"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="text/javascript" http-equiv="content-script-type"/>
<meta content="text/css" http-equiv="content-style-type"/>
<title>レース情報 |  南関東4競馬場｜nankankeiba.com</title>
<meta content="南関東4競馬,浦和競馬,船橋競馬,大井競馬,川崎競馬,リアルタイム情報,データ検索,開催日程,重賞競走,出馬表,オッズ,成績,決定番組,出走登録馬一覧,能力試験結果,持ち時計分析,種牡馬分析,交通アクセス" name="keywords"/>
<meta content="南関東4競馬の開催日程、重賞競走、出馬表、オッズ、成績、決定番組、出走登録馬一覧、能力試験結果、種牡馬分析、持ち時計の分析などの馬券検討情報" name="discription"/>
<link href="/shared/css/nankan_lowerpage_min.css?v=20201101" media="screen,projection" rel="stylesheet" type="text/css"/>
<link class="css_fixed" disabled="" href="/shared/css/_r_fixed_lowerpage_min.css?v=20210618" medi

In [146]:
racename = soup.select_one("span.race-name").text
txs = soup.select_one("p.tx-small").text.split()[0:3]
racetype = " ".join(txs)
# ['サラブレッド系', '一般', '（普通競走）']
hold = soup.select_one("span.tx-mid").text
# re.sub("\u3000|\t", " ", hold)
s = soup.select_one("div#race-data01-a")
time = s.select_one("strong").text
course = re.sub("\n", "", s.select_one("a").text)
print(racename, racetype, course, time)

 Ｃ３(三)(四)　 サラブレッド系 一般 （普通競走） ダ1,400m（外） 発走時刻 12:10


In [158]:
url = "https://keiba.rakuten.co.jp/jockey/urawa"
soup = get_soup(url)
soup.select("div.jockeyList_img")

[<div class="jockeyList_img">
 <img alt="赤津 和希" onerror="this.src='https://keiba.r10s.jp/ts_img/jockey/jockey_profile/img-profile-noimage.png?id=202107121715'; this.alt=''" src="https://keiba.r10s.jp/ts_img/jockey/jockey_profile/31266.jpg?id=202107121715"/>
 </div>,
 <div class="jockeyList_img">
 <img alt="秋元 耕成" onerror="this.src='https://keiba.r10s.jp/ts_img/jockey/jockey_profile/img-profile-noimage.png?id=202107121715'; this.alt=''" src="https://keiba.r10s.jp/ts_img/jockey/jockey_profile/30982.jpg?id=202107121715"/>
 </div>,
 <div class="jockeyList_img">
 <img alt="内田 利雄" onerror="this.src='https://keiba.r10s.jp/ts_img/jockey/jockey_profile/img-profile-noimage.png?id=202107121715'; this.alt=''" src="https://keiba.r10s.jp/ts_img/jockey/jockey_profile/30145.jpg?id=202107121715"/>
 </div>,
 <div class="jockeyList_img">
 <img alt="岡田 大" onerror="this.src='https://keiba.r10s.jp/ts_img/jockey/jockey_profile/img-profile-noimage.png?id=202107121715'; this.alt=''" src="https://keiba.r10s.jp/

In [94]:
odds_url = "https://www.nankankeiba.com/odds/202203171813041101.do"
dfs = get_dfs(odds_url)
dfs[-4]

,枠番,馬番,馬名,単勝,複勝
0,1,1,ラインブレイカー,9.2,2.1-3.4
1,2,2,クールフォルテ,5.5,1.4-2.2
2,3,3,スティールルージュ,3.6,1.2-1.7
3,4,4,ヒストリックノヴァ,5.8,1.6-2.5
4,5,5,プラチナプライド,17.1,2.4-4.0
5,6,6,エミーブレイズ,競走除外,−
6,6,7,トキノゴールド,43.9,5.6-9.8
7,7,8,ティーズハクア,62.9,6.7-11.8
8,7,9,ロマンスロード,42.4,6.1-10.8
9,8,10,ピンクプラム,153.9,14.5-25.7


In [156]:
date = "0317"
course = "浦和"
hold_code = "1304"
hold = date + course_d[course] + hold_code

race_url = url + "/program/2022" + hold + ".do"
soup = get_soup(race_url)
meta = soup.select_one("div.bl-left").text
meta = re.sub("\n|\r", "", meta)
re.sub("\u3000", " ", meta)

'第13回 浦和競馬 第4日 2022年3月17日'

In [72]:
date = "0317"
course = "18"
hold = "1304"
url = "https://www.nankankeiba.com/program/2022" + date + course + hold + ".do"
dfs = get_dfs(url)
dfs[0]

,レース,発走時刻,出走表,出走表.1,オッズ,結果,変更
0,NaN,11:10,Ｃ３４歳ウ,NaN,NaN,NaN,NaN
1,NaN,11:40,３歳(八)選抜馬ウ,NaN,NaN,NaN,NaN
2,NaN,12:10,Ｃ３(三)(四),NaN,NaN,NaN,NaN
3,NaN,12:40,桜のまち幸手賞 Ｃ２(八)(九),NaN,NaN,NaN,NaN
4,NaN,13:10,浦和８００ラウンド Ｃ２,NaN,NaN,NaN,NaN
5,NaN,13:40,浦和８００ラウンド Ｃ２,NaN,NaN,NaN,NaN
6,NaN,14:15,Ｃ３(三)(四),NaN,NaN,NaN,NaN
7,NaN,14:50,桜区の花 サクラソウ賞 Ｃ２(八)(九),NaN,NaN,NaN,NaN
8,NaN,15:25,春の夢（はるのゆめ）特別 Ｃ１(二),NaN,NaN,NaN,NaN
9,NaN,16:00,春和（しゅんわ）特別 Ｂ３(三),NaN,NaN,NaN,NaN


In [73]:
race = "11"
url = "https://www.nankankeiba.com/uma_shosai/2022" + date + course + hold + race + ".do"
res = requests.get(url, headers=ua)
soup = BeautifulSoup(res.content, "html.parser")
tables = soup.select("table")

In [74]:
tags = tables[-1].select("a[href^='/uma_info']")
tags[0].get("href")

'/uma_info/2019105488.do'

In [67]:
url = "https://www.nankankeiba.com/uma_shosai/2022" + date + course + hold + race + ".do"
dfs = get_dfs(url)
#dfs[0]

In [68]:
df = dfs[1]

In [69]:
tpls = [t for t in df.itertuples()]
for i, t in enumerate(tpls[0]):
    print(i, t)

0 0
1 1
2 1
3 リアルインパクト  ラインブレイカー  [大井] 牝3 栗毛  ラインダール  （クロフネ）  吉田勝己
4 今野忠  (川崎)  54.0  荒山勝  (大井)
5 writeOdds(1);
6 -  -
7 472  ｜  491
8 大1286稍ダ 2  --  大1427良ダ 1  --  --
9 1  1  0  0  --  1  0  0  1  --  --
10 --  --  2  2  0  1  --  2  2  0  1
11 1  大井22.1.25 良内ダ 1600  ’２２桃花賞  15頭4番6人 今野忠 54.0  491k クールフォルテ  1427 (0.1) 1-1-1-1 39.7
12 11  大井21.12.31 良内ダ 1600  東京２歳優駿牝馬（ＳI）  16頭3番10人 今野忠 54.0  494k スピーディキック  1446 (1.8) 5-6-3-4 41.2
13 2  大井21.11.4 稍外ダ 1400  ２歳 はくたか特別  16頭8番4人 本橋孝 54.0  482k ロマンスロード  1286 (0.2) 4-4-3 38.7
14 1  大井21.7.1 不外ダ 1400  ２歳７２．５  9頭5番1人 藤田凌 54.0  476k コブラ  1290 (1.5) 1-1-1 39.7
15 2  大井21.6.11 良外ダ 1200  ２歳新馬  6頭3番2人 藤田凌 54.0  472k ママママカロニ  1145 (0.4) 2-2 37.3


In [24]:
tpls[0][11]

'3  浦和22.2.21 良外ダ 1400  ３歳(三)  9頭4番7人 町田直 54.0  455k コンアモーレ  1326 (1.5) 2-2-2-3 40.8'

In [78]:
url = "https://keiba.rakuten.co.jp/jockey/urawa"
res = requests.get(url, headers=ua)
soup = BeautifulSoup(res.content, "html.parser")
soup

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">

<html lang="ja" prefix="og: http://ogp.me/ns# fb: http://www.facebook.com/2008/fbml mixi: http://mixi-platform.com/ns# gr: http://gree.jp/ns">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="telephone=no" name="format-detection"/>
<meta content="浦和所属騎手一覧 ：楽天競馬" name="title"/>
<meta content="浦和所属騎手一覧では、浦和競馬場に所属している騎手をご紹介します。" name="description"/>
<meta content="浦和,浦和競馬場,騎手一覧,ジョッキー一覧,所属一覧" name="keywords"/>
<meta content="ja" name="language"/>
<meta content="width=1020" name="viewport"/>
<meta content="浦和所属騎手一覧 ：楽天競馬" property="og:title"/>
<meta content="浦和所属騎手一覧では、浦和競馬場に所属している騎手をご紹介します。" property="og:description"/>
<meta content="楽天競馬" property="og:site_name"/>
<meta content="https://keiba.r10s.jp/img/ogp/ogp_jockey_list.jpg?id=cfc1d6ddbb6f6a3bac31" property="og:image"/>
<meta content="website" property="og:type"/>
<meta content="277381235708

In [85]:
tags = soup.select(".jockeyList_img img")
tags[0].get("src")

'https://keiba.r10s.jp/ts_img/jockey/jockey_profile/31266.jpg?id=202107121715'